<a href="https://colab.research.google.com/github/shranith/ML-notebooks/blob/master/text_classification_20ng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a text classification model with TF Hub

In this notebook, we'll walk you through building a model to predict the genres of a movie given its description. The emphasis here is not on accuracy, but instead how to use TF Hub layers in a text classification model.

To start, import the necessary dependencies for this project.

In [11]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import json
import pickle
import urllib

from sklearn.preprocessing import LabelEncoder

print(tf.__version__)

1.12.0-rc2


In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
!ls
!ls gdrive

gdrive	sample_data
'My Drive'


In [16]:
data = pd.read_csv('/content/gdrive/My Drive/20ng/20news-bydate-train.csv')
print(data.head())

descriptions = data['text']
category = data['category']


category[:10]


   text_id                                               text  \
0     5701  From: nrmendel@unix.amherst.edu (Nathaniel Men...   
1     1387  From: orly@phakt.usc.edu (Mr. Nitro Plastique)...   
2     8953  From: hays@ssd.intel.com (Kirk Hays)\n Subject...   
3     2880  From: rnichols@cbnewsg.cb.att.com (robert.k.ni...   
4     4348  From: steve-b@access.digex.com (Steve Brinich)...   

                  category  
0          rec.motorcycles  
1    comp.sys.mac.hardware  
2       talk.politics.guns  
3  comp.os.ms-windows.misc  
4                sci.crypt  


0            rec.motorcycles
1      comp.sys.mac.hardware
2         talk.politics.guns
3    comp.os.ms-windows.misc
4                  sci.crypt
5                alt.atheism
6             comp.windows.x
7                alt.atheism
8                  sci.crypt
9                  sci.crypt
Name: category, dtype: object

## The dataset

We need a lot of text inputs to train our model. For this model we'll use [this awesome movies dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset) from Kaggle. To simplify things I've made the `movies_metadata.csv` file available in a public Cloud Storage bucket so we can download it with `wget`. I've preprocessed the dataset already to limit the number of genres we'll use for our model, but first let's take a look at the original data so we can see what we're working with.

Next we'll convert the dataset to a Pandas dataframe and print the first 5 rows. For this model we're only using 2 of these columns: `genres` and `overview`.

## Preparing the data for our model

I've done some preprocessing to limit the dataset to the top 9 genres, and I've saved the Pandas dataframes as public [Pickle](https://docs.python.org/3/library/pickle.html) files in GCS. Here we download those files. The resulting `descriptions` and `genres` variables are Pandas Series containing all descriptions and genres from our dataset respectively.

In [17]:
type(descriptions)
descriptions[:10]

print(type(category[1]))
type(category)


<class 'str'>


pandas.core.series.Series

### Splitting our data
When we train our model, we'll use 80% of the data for training and set aside 20% of the data to evaluate how our model performed.

In [0]:
train_size = int(len(descriptions) * .8)

train_descriptions = descriptions[:train_size].astype('str')
train_genres = category[:train_size]

test_descriptions = descriptions[train_size:].astype('str')
test_genres = category[train_size:]

In [19]:
print(test_genres)

9016      comp.os.ms-windows.misc
9017                  alt.atheism
9018              rec.motorcycles
9019       soc.religion.christian
9020              rec.motorcycles
9021      comp.os.ms-windows.misc
9022             rec.sport.hockey
9023                      sci.med
9024      comp.os.ms-windows.misc
9025                 misc.forsale
9026      comp.os.ms-windows.misc
9027              sci.electronics
9028                    sci.space
9029             rec.sport.hockey
9030                      sci.med
9031              rec.motorcycles
9032     comp.sys.ibm.pc.hardware
9033      comp.os.ms-windows.misc
9034     comp.sys.ibm.pc.hardware
9035      comp.os.ms-windows.misc
9036           talk.politics.guns
9037      comp.os.ms-windows.misc
9038     comp.sys.ibm.pc.hardware
9039              sci.electronics
9040              sci.electronics
9041                      sci.med
9042       soc.religion.christian
9043                    sci.space
9044                    sci.space
9045          

### Formatting our labels
When we train our model we'll provide the labels (in this case genres) associated with each movie. We can't pass the genres in as strings directly, we'll transform them into multi-hot vectors. Since we have 9 genres, we'll have a 9 element vector for each movie with 0s and 1s indicating which genres are present in each description.

In [26]:
# print(train_genres)


encoder = LabelEncoder()
encoder.fit_transform(train_genres)
train_encoded = encoder.transform(train_genres)
test_encoded = encoder.transform(test_genres)
num_classes = len(encoder.classes_)

# Print all possible genres and the labels for the first movie in our training dataset
print(encoder.classes_)
print(train_encoded[0])

['alt.atheism' 'comp.graphics' 'comp.os.ms-windows.misc'
 'comp.sys.ibm.pc.hardware' 'comp.sys.mac.hardware' 'comp.windows.x'
 'misc.forsale' 'rec.autos' 'rec.motorcycles' 'rec.sport.baseball'
 'rec.sport.hockey' 'sci.crypt' 'sci.electronics' 'sci.med' 'sci.space'
 'soc.religion.christian' 'talk.politics.guns' 'talk.politics.mideast'
 'talk.politics.misc' 'talk.religion.misc']
8


### Create our TF Hub embedding layer
[TF Hub]() provides a library of existing pre-trained model checkpoints for various kinds of models (images, text, and more) In this model we'll use the TF Hub `universal-sentence-encoder` module for our pre-trained word embeddings. We only need one line of code to instantiate module. When we train our model, it'll convert our array of movie description strings to embeddings. When we train our model, we'll use this as a feature column.


In [22]:
description_embeddings = hub.text_embedding_column("descriptions", module_spec="https://tfhub.dev/google/universal-sentence-encoder/2", trainable=False)


INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.


## Instantiating our DNNEstimator Model
The first parameter we pass to our DNNEstimator is called a head, and defines the type of labels our model should expect. Since we want our model to output multiple labels, we’ll use multi_label_head here. Then we'll convert our features and labels to numpy arrays and instantiate our Estimator. `batch_size` and `num_epochs` are hyperparameters - you should experiment with different values to see what works best on your dataset.

In [0]:
multi_label_head = tf.contrib.estimator.multi_class_head(
    num_classes,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)

In [28]:
features = {
  "descriptions": np.array(train_descriptions).astype(np.str)
}
labels = np.array(train_encoded).astype(np.int32)
train_input_fn = tf.estimator.inputs.numpy_input_fn(features, labels, shuffle=True, batch_size=32, num_epochs=25)
estimator = tf.contrib.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64,10],
    feature_columns=[description_embeddings])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpyijh07th', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2028f8e048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training and serving our model 
To train our model, we simply call `train()` passing it the input function we defined above. Once our model is trained, we'll define an evaluation input function similar to the one above and call `evaluate()`. When this completes we'll get a few metrics we can use to evaluate our model's accuracy.


In [29]:
estimator.train(input_fn=train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpyijh07th/model.ckpt.
INFO:tensorflow:loss = 2.9876974, step = 0
INFO:tensorflow:global_step/sec: 12.7507
INFO:tensorflow:loss = 2.9261374, step = 100 (7.850 sec)
INFO:tensorflow:global_step/sec: 12.4862
INFO:tensorflow:loss = 2.6610343, step = 200 (8.004 sec)
INFO:tensorflow:global_step/sec: 12.5786
INFO:tensorflow:loss = 1.9961158, step = 300 (7.950 sec)
INFO:tensorflow:global_step/sec: 12.7406
INFO:tensorflow:loss = 1.7117159, step = 400 (7.8

In [30]:
# Define our eval input_fn and run eval
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(test_descriptions).astype(np.str)}, test_encoded.astype(np.int32), shuffle=False)
estimator.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-31-06:09:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpyijh07th/model.ckpt-7044
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-31-06:09:57
INFO:tensorflow:Saving dict for global step 7044: accuracy = 0.71472937, average_loss = 0.88901, global_step = 7044, loss = 0.88745
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 7044: /tmp/tmpyijh07th/model.ckpt-7044


{'accuracy': 0.71472937,
 'average_loss': 0.88901,
 'global_step': 7044,
 'loss': 0.88745}

## Generating predictions on new data
Now for the most fun part! Let's generate predictions on movie descriptions our model hasn't seen before. We'll define an array of 3 new description strings (the comments indicate the correct genres) and create a `predict_input_fn`. Then we'll display the top 2 genres along with their confidence percentages for each of the 3 movies.

In [0]:
# Test our model on some raw description data
raw_test = [
    "An examination of our dietary choices and the food we put in our bodies. Based on Jonathan Safran Foer's memoir.", # Documentary
    "After escaping an attack by what he claims was a 70-foot shark, Jonas Taylor must confront his fears to save those trapped in a sunken submersible.", # Action, Adventure
    "A teenager tries to survive the last week of her disastrous eighth-grade year before leaving to start high school.", # Comedy
]



In [0]:
# Generate predictions
predict_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(raw_test).astype(np.str)}, shuffle=False)
results = estimator.predict(predict_input_fn)

In [33]:
# Display predictions
for movie_genres in results:
  top_2 = movie_genres['probabilities'].argsort()[-2:][::-1]
  for genre in top_2:
    text_genre = encoder.classes_[genre]
    print(text_genre + ': ' + str(round(movie_genres['probabilities'][genre] * 100, 2)) + '%')
  print('')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpyijh07th/model.ckpt-7044
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sci.med: 95.14%
soc.religion.christian: 3.24%

rec.sport.hockey: 40.49%
sci.space: 23.56%

misc.forsale: 30.8%
rec.motorcycles: 28.38%

